In [22]:
import numpy as np
import os
from scipy.io import wavfile
import librosa
import csv
import math as m
from natsort import natsorted
import h5py
from sklearn import preprocessing
#in this section the features for mixed voices are extracted.
# orig_path=os.getcwd()
i = 1
Data_path = 'D:/studies/university/thesis/speech_separation_codes/du16/donesomestuff'
mixed_folder = os.path.normpath(os.path.join(Data_path,'lilmixed'))
a = os.listdir(os.path.join(mixed_folder))
a = natsorted(a)
# f.close()
mixed_logname = 'mixed_log_3h_norm'
mixed_phasename = 'mixed_phase_3h_norm'
mixed_logpath = os.path.normpath(os.path.join(Data_path,mixed_logname))
mixed_phasepath = os.path.normpath(os.path.join(Data_path,mixed_phasename))

f = h5py.File(mixed_logpath+'.hdf5', 'w')
d = f.create_dataset(mixed_logname, (0,257),maxshape=(None,257), dtype='f', chunks=True)
# ff.close()
ff = h5py.File(mixed_phasepath+'.hdf5', 'w')
dd = ff.create_dataset(mixed_phasename, (0,257),maxshape=(None,257), dtype='f', chunks=True)
#################################
w=3
#################################
for filename in a:
    wav_data, sr = librosa.load(os.path.join(mixed_folder,filename), sr=16000) 
    framed_data=librosa.core.stft(wav_data, n_fft=512, hop_length=256, win_length=512, window='hann')
    print(framed_data.shape)
    abslt=np.absolute(framed_data)**2
    dft_signal=np.log10(abslt+1e-7*np.ones(np.shape(abslt)))
    dft_signal=preprocessing.scale(dft_signal, axis=0, with_mean=True, with_std=True, copy=True)
    for i in range(m.floor(w/2)):
        dft_signal=np.insert(dft_signal,0,0,axis = 1)
    data_phase=np.angle(framed_data)
    for i in range(m.floor(w/2)):
        data_phase=np.insert(data_phase,0,0,axis = 1)
    d.resize(d.shape[0]+dft_signal.shape[1], axis=0)   
    d[-1*dft_signal.shape[1]:] = dft_signal.T
    dd.resize(dd.shape[0]+data_phase.shape[1], axis=0)   
    dd[-1*data_phase.shape[1]:] = data_phase.T
    i=i+1
####################################################
####################################################
import h5py
import math as m
import os
import numpy as np
import librosa
from natsort import natsorted
orig_path=os.getcwd()
wav=[]
clean_raw = 'single_dataset_16'
l=os.listdir(os.path.join(Data_path,clean_raw))
l=natsorted(l)
filename1 = l[1]
i=0
###################################################
w=3
###################################################
num_of_snrs=6
num_of_speakers=2
files = os.listdir(os.path.join(Data_path,clean_raw,filename1))
files=natsorted(files)
f.close()
ff.close()
clean_log = 'single_dataset_log_3h_norm'
clean_phase = 'single_dataset_phase_3h_norm'
clean_log_path = os.path.normpath(os.path.join(Data_path,clean_log))
clean_phase_path = os.path.normpath(os.path.join(Data_path,clean_phase))
f = h5py.File(clean_log_path+'.hdf5', 'w')
d = f.create_dataset(clean_log, (0,257),maxshape=(None,257), dtype='f', chunks=True)
ff = h5py.File(clean_phase_path+'.hdf5', 'w')
dd = ff.create_dataset(clean_phase, (0,257),maxshape=(None,257), dtype='f', chunks=True)
#in this section the features for clean voices are extracted.
for voice in files:
    wav_data, sr = librosa.load(os.path.join(Data_path,clean_raw,filename1,voice), sr=16000) 
    framed_data=librosa.core.stft(wav_data, n_fft=512, hop_length=256, win_length=512, window='hann')
    print(framed_data.shape)
    abslt=np.absolute(framed_data)**2
    dft_signal=np.log10(abslt+1e-7*np.ones(np.shape(abslt)))
    data_phase=np.angle(framed_data)
#     for i in range(m.floor(w/2)):
#         dft_signal=np.insert(dft_signal,0,0,axis = 1)
    for i in range(num_of_snrs*num_of_speakers):
        d.resize(d.shape[0]+dft_signal.shape[1], axis=0)   
        d[-1*dft_signal.shape[1]:] = dft_signal.T
        dd.resize(dd.shape[0]+data_phase.shape[1], axis=0)   
        dd[-1*data_phase.shape[1]:] = data_phase.T
###############################################
###############################################
import math as m
import numpy as np
import h5py
##############################################
w = 3
###############################################
mixed_logname='mixed_log_3h_norm'
mixed_phasename='mixed_phase_3h_norm'
mixed_log_path = os.path.normpath(os.path.join(Data_path,mixed_logname))
clean_phase_path = os.path.normpath(os.path.join(Data_path,mixed_phasename))
import h5py
h5f = h5py.File(mixed_log_path+'.hdf5','r')
ftr = h5f[mixed_logname][0:]
# h5f.close()
ftr_refrmd=[]
print('0')
for i in range(m.floor(w/2)):
    p=-m.floor(w/2)
    temp = []
    for j in range(w):
        if np.all(ftr[i]==0):
            break
        temp.extend(ftr[i+p+j])
    if np.any(temp!=[]):
        ftr_refrmd.append(temp)
# f.close()
print('1')
refrmd_file='ftr_refrmd_3h_norm'
refrmd_file_path = os.path.normpath(os.path.join(Data_path,refrmd_file))
f = h5py.File(refrmd_file_path+'.hdf5', 'w')
d = f.create_dataset(refrmd_file, (0,257*w),maxshape=(None,257*w), dtype='f', chunks=True)
print('2')
for i in range(m.floor(w/2),ftr.shape[0]-m.floor(w/2)-1):
    k=-m.floor(w/2)
    temp = []
    for j in range(w):
        if np.all(ftr[i]==0):
            break
        temp.extend(ftr[i+k])
        k=k+1
    if np.any(temp!=[]):
        ftr_refrmd.append(temp)
    if len(ftr_refrmd)>50000 or i==ftr.shape[0]-m.floor(w/2)-2:
        d.resize(d.shape[0]+len(ftr_refrmd), axis=0)   
        d[-1*len(ftr_refrmd):] = ftr_refrmd
        ftr_refrmd=[]
    if i%1000==0:
        print(i)
for i in range(ftr.shape[0]-m.floor(w/2)-1,ftr.shape[0]):
    p=-m.floor(w/2)
    temp = []
    for j in range(w):
        if np.all(ftr[i]==0):
            break
        if j>m.floor(w/2)+(ftr.shape[0]-i)-1:
            temp.extend(np.zeros(ftr[ftr.shape[0]-1].shape))
        else:
            temp.extend(ftr[i+p+j])
    if np.any(temp!=[]):
        ftr_refrmd.append(temp)
d.resize(d.shape[0]+len(ftr_refrmd), axis=0)   
d[-1*len(ftr_refrmd):] = ftr_refrmd
ftr_refrmd=[]
f.close()
h5f.close()
##############################################
##############################################
# import h5py
# import numpy as np
# h5 = h5py.File(refrmd_file, 'r')
# d=h5[refrmd_file]
# tot_len=d.shape[0]
# lst=0
# h5.close()
# on=0
# h5f = h5py.File(refrmd_file+'.hdf5','r')
# for i in range(0,tot_len,1000):
#     mixed = h5f[refrmd_file][i:i+1000]
#     lst=lst+np.sum(mixed,axis=0)
#     on=on+len(mixed)
# tot_mean=lst/tot_len
# h5f.close()
# lst=0
# h5f = h5py.File(refrmd_file+'.hdf5','r')
# for i in range(0,tot_len,1000):
#     mixed = h5f[refrmd_file][i:i+1000]
#     lst=lst+np.sum((mixed - tot_mean)**2,axis=0)
# lst=lst/tot_len
# tot_std=np.sqrt(lst)
# h5f.close()
# f = h5py.File('ftr_scaled.hdf5', 'w')
# dd = f.create_dataset('ftr_scaled', (0,1799),maxshape=(None,1799), dtype='f', chunks=True)
# h5f = h5py.File('ftr_refrmd.hdf5','r')
# for i in range(0,tot_len,1000):
#     mixed = h5f['ftr_refrmd'][i:i+1000]
#     scaled=(mixed-tot_mean)/tot_std
#     dd.resize(dd.shape[0]+scaled.shape[0], axis=0)   
#     dd[-1*scaled.shape[0]:] = scaled
# np.savetxt('mean.txt',tot_mean)
# np.savetxt('std.txt',tot_std)

(257, 102)
(257, 104)
(257, 105)
(257, 111)
(257, 97)
(257, 117)
(257, 114)
(257, 111)
(257, 117)
(257, 122)
(257, 113)
(257, 108)
(257, 112)
(257, 116)
(257, 120)
(257, 101)
(257, 114)
(257, 108)
(257, 101)
(257, 91)
(257, 100)
(257, 116)
(257, 127)
(257, 94)
(257, 112)
(257, 105)
(257, 106)
(257, 106)
(257, 109)
(257, 108)
(257, 109)
(257, 111)
(257, 112)
(257, 109)
(257, 103)
(257, 111)
(257, 131)
(257, 103)
(257, 108)
(257, 109)
(257, 108)
(257, 106)
(257, 104)
(257, 114)
(257, 128)
(257, 115)
(257, 102)
(257, 122)
(257, 101)
(257, 109)
(257, 122)
(257, 114)
(257, 107)
(257, 107)
(257, 104)
(257, 109)
(257, 119)
(257, 107)
(257, 109)
(257, 107)
(257, 115)
(257, 109)
(257, 109)
(257, 114)
(257, 114)
(257, 111)
(257, 119)
(257, 119)
(257, 119)
(257, 102)
(257, 101)
(257, 98)
(257, 102)
(257, 112)
(257, 107)
(257, 96)
(257, 112)
(257, 106)
(257, 110)
(257, 101)
(257, 105)
(257, 111)
(257, 113)
(257, 146)
(257, 114)
(257, 105)
(257, 132)
(257, 111)
(257, 101)
(257, 102)
(257, 107)
(257

In [1]:
from sklearn import preprocessing
a=[[1,2,3,4],[2,3,6,2],[1,4,6,23],[1,34,55,6]]
b=preprocessing.scale(a, axis=0, with_mean=True, with_std=True, copy=True)

In [4]:
import numpy as np
np.mean(b[:,0])

-5.551115123125783e-17

In [21]:
clean_log = 'single_dataset_log_3h_norm'
clean_phase = 'single_dataset_phase_3h_norm'
clean_log_path = os.path.normpath(os.path.join(Data_path,clean_log))
clean_phase_path = os.path.normpath(os.path.join(Data_path,clean_phase))
f = h5py.File(clean_log+'.hdf5', 'w')
d = f.create_dataset(clean_log_path, (0,257),maxshape=(None,257), dtype='f', chunks=True)
ff = h5py.File(clean_phase+'.hdf5', 'w')
dd = ff.create_dataset(clean_phase_path, (0,257),maxshape=(None,257), dtype='f', chunks=True)

In [6]:
# ff.close()
f.close()

NameError: name 'f' is not defined

In [30]:
import numpy as np
import tensorflow as tf
import os
__author__ = "Sangwoong Yoon"

def np_to_tfrecords(X, Y, file_path_prefix, verbose=True):
    """
    Converts a Numpy array (or two Numpy arrays) into a tfrecord file.
    For supervised learning, feed training inputs to X and training labels to Y.
    For unsupervised learning, only feed training inputs to X, and feed None to Y.
    The length of the first dimensions of X and Y should be the number of samples.
    
    Parameters
    ----------
    X : numpy.ndarray of rank 2
        Numpy array for training inputs. Its dtype should be float32, float64, or int64.
        If X has a higher rank, it should be rshape before fed to this function.
    Y : numpy.ndarray of rank 2 or None
        Numpy array for training labels. Its dtype should be float32, float64, or int64.
        None if there is no label array.
    file_path_prefix : str
        The path and name of the resulting tfrecord file to be generated, without '.tfrecords'
    verbose : bool
        If true, progress is reported.
    
    Raises
    ------
    ValueError
        If input type is not float (64 or 32) or int.
    
    """
    def _dtype_feature(ndarray):
        """match appropriate tf.train.Feature class with dtype of ndarray. """
        assert isinstance(ndarray, np.ndarray)
        dtype_ = ndarray.dtype
        if dtype_ == np.float64 or dtype_ == np.float32:
            return lambda array: tf.train.Feature(float_list=tf.train.FloatList(value=array))
        elif dtype_ == np.int64:
            return lambda array: tf.train.Feature(int64_list=tf.train.Int64List(value=array))
        else:  
            raise ValueError("The input should be numpy ndarray. \
                               Instaed got {}".format(ndarray.dtype))
            
    assert isinstance(X, np.ndarray)
    assert len(X.shape) == 2  # If X has a higher rank, 
                               # it should be rshape before fed to this function.
    assert isinstance(Y, np.ndarray) or Y is None
    
    # load appropriate tf.train.Feature class depending on dtype
    dtype_feature_x = _dtype_feature(X)
    if Y is not None:
        assert X.shape[0] == Y.shape[0]
        assert len(Y.shape) == 2
        dtype_feature_y = _dtype_feature(Y)            
    
    # Generate tfrecord writer
    result_tf_file = file_path_prefix + '.tfrecords'
    writer = tf.python_io.TFRecordWriter(result_tf_file)
    if verbose:
        print("Serializing {:d} examples into {}".format(X.shape[0], result_tf_file))
        
    # iterate over each sample,
    # and serialize it as ProtoBuf.
    for idx in range(X.shape[0]):
        x = X[idx]
        if Y is not None:
            y = Y[idx]
        
        d_feature = {}
        d_feature['X'] = dtype_feature_x(x)
        if Y is not None:
            d_feature['Y'] = dtype_feature_y(y)
            
        features = tf.train.Features(feature=d_feature)
        example = tf.train.Example(features=features)
        serialized = example.SerializeToString()
        writer.write(serialized)
    
    if verbose:
        print("Writing {} done!".format(result_tf_file))

        
#################################    
##      Test and Use Cases     ##
#################################
import h5py
load_size = 500000
k = 0
Data_path = 'D:/studies/university/thesis/speech_separation_codes/du16/donesomestuff'
input_name = 'ftr_refrmd_3h_norm'
input_path = os.path.normpath(os.path.join(Data_path,input_name))
target_name = 'single_dataset_log_3h_norm'
target_path = os.path.normpath(os.path.join(Data_path,target_name))
h5f1 = h5py.File(input_path+'.hdf5','r')
hh = h5py.File(target_path+'.hdf5', 'r')
d=hh[target_name]
data_len = d.shape[0]
index = np.arange(0, data_len, load_size)
for I in index:
    if data_len-I<load_size:
        xx = h5f1[input_name][I:]
        yy = hh[target_name][I:]
    else:
        xx = h5f1[input_name][I:I+load_size]
        yy = hh[target_name][I:I+load_size]
    np_to_tfrecords(xx, yy, os.path.normpath(os.path.join(Data_path,'tfrecord_files','tfrecord_files_3h_norm','filenum'+str(k))), verbose=True)
    k = k+1
# 1-2. Check if the data is stored correctly
# open the saved file and check the first entries
# for serialized_example in tf.python_io.tf_record_iterator('test.tfrecords'):
#     example = tf.train.Example()
#     example.ParseFromString(serialized_example)
#     x_1 = np.array(example.features.feature['X'].float_list.value)
#     y_1 = np.array(example.features.feature['Y'].float_list.value)
#     break

# np_to_tfrecords(X, Y, file_path_prefix, verbose=True)

Serializing 500000 examples into D:\studies\university\thesis\speech_separation_codes\du16\donesomestuff\tfrecord_files\tfrecord_files_3h_norm\filenum0.tfrecords
Writing D:\studies\university\thesis\speech_separation_codes\du16\donesomestuff\tfrecord_files\tfrecord_files_3h_norm\filenum0.tfrecords done!
Serializing 184108 examples into D:\studies\university\thesis\speech_separation_codes\du16\donesomestuff\tfrecord_files\tfrecord_files_3h_norm\filenum1.tfrecords
Writing D:\studies\university\thesis\speech_separation_codes\du16\donesomestuff\tfrecord_files\tfrecord_files_3h_norm\filenum1.tfrecords done!


In [34]:
for serialized_example in tf.python_io.tf_record_iterator('tfrecord_files/test3.tfrecords'):
    example = tf.train.Example()
    example.ParseFromString(serialized_example)
    x_1 = np.array(example.features.feature['X'].float_list.value)
    y_1 = np.array(example.features.feature['Y'].float_list.value)
    break

In [40]:
len(example.features.feature['X'].float_list.value)

771

In [35]:
#define reconstruct function to reconstruct sound from framed signal.
def reconstruct(wave,angle):
    recon = np.sqrt(np.power(10, wave))
    recon1 = recon*np.cos(angle)+recon*np.sin(angle)*1j
    recon = librosa.core.istft((recon1.T), hop_length=256, win_length=512, window='hann')
    return recon

In [8]:
# ff.close()
# f.close()
h5f.close()

NameError: name 'h5f' is not defined

In [63]:
import h5py
import numpy as np
import librosa
h5f = h5py.File('mixed_log2.hdf5','r')
log = h5f['mixed_log2'][105:209]
h5f.close()
h5f = h5py.File('mixed_phase2.hdf5','r')
phase = h5f['mixed_phase2'][105:209]
h5f.close()
recon_mixed=reconstruct(log, phase)

In [38]:
import librosa
h5f = h5py.File('single_dataset_log_16.hdf5','r')
log_single = h5f['single_dataset_log_16'][0:1000]
h5f.close()
h5f = h5py.File('single_dataset_phase_16.hdf5','r')
phase_single = h5f['single_dataset_phase_16'][0:1000]
h5f.close()
recon_single=reconstruct(log_single, phase_single)

In [45]:
import sounddevice as sd
sd.play(recon_single,16000)

In [44]:
log_single[0]

array([-1.342651 , -2.1882577, -3.7148652, -4.6653256, -5.437377 ,
       -4.43097  , -6.40937  , -5.0770507, -4.1862173, -4.116098 ,
       -6.2492156, -4.275731 , -5.0884843, -6.1064577, -5.4683805,
       -5.730542 , -6.398159 , -6.999886 , -5.9630837, -5.7165947,
       -6.9044456, -6.9755797, -5.9757214, -6.323105 , -6.841815 ,
       -6.559472 , -6.2862096, -6.6157503, -6.8178153, -6.0939927,
       -6.594226 , -6.445905 , -6.1755342, -6.6693892, -6.7243204,
       -5.352902 , -5.3833055, -6.940676 , -5.861181 , -6.998239 ,
       -6.7733436, -6.2383866, -6.469671 , -6.6878834, -6.875888 ,
       -6.880289 , -6.155297 , -6.2254114, -6.172742 , -6.1801596,
       -6.492664 , -6.0942006, -6.999407 , -6.255904 , -6.174611 ,
       -6.992984 , -6.1967263, -6.2247787, -6.1919084, -6.3945003,
       -6.922461 , -6.858297 , -6.984833 , -6.6360354, -6.3415375,
       -6.9706817, -6.441416 , -6.60832  , -6.1045475, -6.2018385,
       -6.8768635, -6.361902 , -6.777529 , -6.954711 , -6.5786

In [46]:
import h5py
h5f = h5py.File('mixed_log2.hdf5','r')
log = h5f['mixed_log2'][0:1000]
h5f.close()
h5ff = h5py.File('ftr_refrmd.hdf5','r')
log3 = h5ff['ftr_refrmd'][0:1000]
h5ff.close()

In [8]:
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

In [33]:
h5f = h5py.File('mixed_log2.hdf5','r')
mix = h5f['mixed_log2'][0:]

array([ 3.4527845 ,  2.838498  ,  1.2287587 ,  1.3377892 , -0.4405885 ,
       -0.8609227 , -0.54041094, -0.5721344 , -0.7892443 , -3.4268441 ,
       -1.0621505 , -0.89862   , -0.93707925, -1.720495  , -2.5171618 ,
       -1.7181373 , -1.7489173 , -1.8884361 , -1.6029108 , -1.4329101 ,
       -1.2100719 , -1.8808707 , -1.7530991 , -2.2131329 , -2.4447558 ,
       -2.1366808 , -3.1237316 , -2.3084223 , -2.6527307 , -3.943142  ,
       -3.4862459 , -3.5423768 , -3.2174733 , -2.9781365 , -2.9799032 ,
       -3.5630665 , -2.5886016 , -2.8071043 , -3.5794282 , -5.2601175 ,
       -3.2601643 , -2.6804316 , -2.3691998 , -2.2037294 , -2.5799422 ,
       -2.8238807 , -5.4355483 , -2.7327795 , -2.3162756 , -2.378921  ,
       -1.9771441 , -1.7113293 , -1.7757478 , -1.3545376 , -0.9116666 ,
       -1.4505057 , -1.6838427 , -1.6365662 , -1.6618577 , -1.4630824 ,
       -1.5523989 , -2.7806113 , -1.9827592 , -1.2144945 , -0.98019284,
       -2.240224  , -1.0997478 , -0.9419252 , -0.93433976, -1.10

In [32]:
log3[291,257*4:257*5]

array([ 3.4527845 ,  2.838498  ,  1.2287587 ,  1.3377892 , -0.4405885 ,
       -0.8609227 , -0.54041094, -0.5721344 , -0.7892443 , -3.4268441 ,
       -1.0621505 , -0.89862   , -0.93707925, -1.720495  , -2.5171618 ,
       -1.7181373 , -1.7489173 , -1.8884361 , -1.6029108 , -1.4329101 ,
       -1.2100719 , -1.8808707 , -1.7530991 , -2.2131329 , -2.4447558 ,
       -2.1366808 , -3.1237316 , -2.3084223 , -2.6527307 , -3.943142  ,
       -3.4862459 , -3.5423768 , -3.2174733 , -2.9781365 , -2.9799032 ,
       -3.5630665 , -2.5886016 , -2.8071043 , -3.5794282 , -5.2601175 ,
       -3.2601643 , -2.6804316 , -2.3691998 , -2.2037294 , -2.5799422 ,
       -2.8238807 , -5.4355483 , -2.7327795 , -2.3162756 , -2.378921  ,
       -1.9771441 , -1.7113293 , -1.7757478 , -1.3545376 , -0.9116666 ,
       -1.4505057 , -1.6838427 , -1.6365662 , -1.6618577 , -1.4630824 ,
       -1.5523989 , -2.7806113 , -1.9827592 , -1.2144945 , -0.98019284,
       -2.240224  , -1.0997478 , -0.9419252 , -0.93433976, -1.10

In [2]:
import math as m
import numpy as np
import h5py
w = 7
import h5py
h5f = h5py.File('mixed_log2.hdf5','r')
ftr = h5f['mixed_log2'][0:]
h5f.close()
ftr_refrmd=[]
for i in range(m.floor(w/2)):
    p=-m.floor(w/2)
    temp = []
    for j in range(w):
        if np.all(ftr[i]==0):
            break
        temp.extend(ftr[i+p+j])
    if np.any(temp!=[]):
        ftr_refrmd.append(temp)
f.close()
f = h5py.File('ftr_refrmd.hdf5', 'w')
d = f.create_dataset('ftr_refrmd', (0,257*7),maxshape=(None,257*7), dtype='f', chunks=True)
for i in range(m.floor(w/2),ftr.shape[0]-m.floor(w/2)-1):
    k=-m.floor(w/2)
    temp = []
    for j in range(w):
        if np.all(ftr[i]==0):
            break
        temp.extend(ftr[i+k])
        k=k+1
    if np.any(temp!=[]):
        ftr_refrmd.append(temp)
    if len(ftr_refrmd)>100000 or i==ftr.shape[0]-m.floor(w/2)-2:
        d.resize(d.shape[0]+len(ftr_refrmd), axis=0)   
        d[-1*len(ftr_refrmd):] = ftr_refrmd
        ftr_refrmd=[]
    if i%1000==0:
        print(i)
for i in range(ftr.shape[0]-m.floor(w/2)-1,ftr.shape[0]):
    p=-m.floor(w/2)
    temp = []
    for j in range(w):
        if np.all(ftr[i]==0):
            break
        if j>m.floor(w/2)+(ftr.shape[0]-i)-1:
            temp.extend(np.zeros(ftr[ftr.shape[0]-1].shape))
        else:
            temp.extend(ftr[i+p+j])
    if np.any(temp!=[]):
        ftr_refrmd.append(temp)
d.resize(d.shape[0]+len(ftr_refrmd), axis=0)   
d[-1*len(ftr_refrmd):] = ftr_refrmd
ftr_refrmd=[]
##############################################
##############################################


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
15

1166000
1167000
1168000
1169000
1170000
1171000
1172000
1173000
1174000
1175000
1176000
1177000
1178000
1179000
1180000
1181000
1182000
1183000
1184000
1185000
1186000
1187000
1188000
1189000
1190000
1191000
1192000
1193000
1194000
1195000
1196000
1197000
1198000
1199000
1200000
1201000
1202000
1203000
1204000
1205000
1206000
1207000
1208000
1209000
1210000
1211000
1212000
1213000
1214000
1215000
1216000
1217000
1218000
1219000
1220000
1221000
1222000
1223000
1224000
1225000
1226000
1227000
1228000
1229000
1230000
1231000
1232000
1233000
1234000
1235000
1236000
1237000
1238000
1239000
1240000
1241000
1242000
1243000
1244000
1245000
1246000
1247000
1248000
1249000
1250000
1251000
1252000
1253000
1254000
1255000
1256000
1257000
1258000
1259000
1260000
1261000
1262000
1263000
1264000
1265000
1266000
1267000
1268000
1269000
1270000
1271000
1272000
1273000
1274000
1275000
1276000
1277000
1278000
1279000
1280000
1281000
1282000
1283000
1284000
1285000
1286000
1287000
1288000
1289000
1290000


2193000
2194000
2195000
2196000
2197000
2198000
2199000
2200000
2201000
2202000
2203000
2204000
2205000
2206000
2207000
2208000
2209000
2210000
2211000
2212000
2213000
2214000
2215000
2216000
2217000
2218000
2219000
2220000
2221000
2222000
2223000
2224000
2225000
2226000
2227000
2228000
2229000
2230000
2231000
2232000
2233000
2234000
2235000
2236000
2237000
2238000
2239000
2240000
2241000
2242000
2243000
2244000
2245000
2246000
2247000
2248000
2249000
2250000
2251000
2252000
2253000
2254000
2255000
2256000
2257000
2258000
2259000
2260000
2261000
2262000
2263000
2264000
2265000
2266000
2267000
2268000
2269000
2270000
2271000
2272000
2273000
2274000
2275000
2276000
2277000
2278000
2279000
2280000
2281000
2282000
2283000
2284000
2285000
2286000
2287000
2288000
2289000
2290000
2291000
2292000
2293000
2294000
2295000
2296000
2297000
2298000
2299000
2300000
2301000
2302000
2303000
2304000
2305000
2306000
2307000
2308000
2309000
2310000
2311000
2312000
2313000
2314000
2315000
2316000
2317000


3218000
3219000
3220000
3221000
3222000
3223000
3224000
3225000
3226000
3227000
3228000
3229000
3230000
3231000
3232000
3233000
3234000
3235000
3236000
3237000
3238000
3239000
3240000
3241000
3242000
3243000
3244000
3245000
3246000
3247000
3248000
3249000
3250000
3251000
3252000
3253000
3254000
3255000
3256000
3257000
3258000
3259000
3260000
3261000
3262000
3263000
3264000
3265000
3266000
3267000
3268000
3269000
3270000
3271000
3272000
3273000
3274000
3275000
3276000
3277000
3278000
3279000
3280000
3281000
3282000
3283000
3284000
3285000
3286000
3287000
3288000
3289000
3290000
3291000
3292000
3293000
3294000
3295000
3296000
3297000
3298000
3299000
3300000
3301000
3302000
3303000
3304000
3305000
3306000
3307000
3308000
3309000
3310000
3311000
3312000
3313000
3314000
3315000
3316000
3317000
3318000
3319000
3320000
3321000
3322000
3323000
3324000
3325000
3326000
3327000
3328000
3329000
3330000
3331000
3332000
3333000
3334000
3335000
3336000
3337000
3338000
3339000
3340000
3341000
3342000


4244000
4245000
4246000
4247000
4248000
4249000
4250000
4251000
4252000
4253000
4254000
4255000
4256000
4257000
4258000
4259000
4260000
4261000
4262000
4263000
4264000
4265000
4266000
4267000
4268000
4269000
4270000
4271000
4272000
4273000
4274000
4275000
4276000
4277000
4278000
4279000
4280000
4281000
4282000
4283000
4284000
4285000
4286000
4287000
4288000
4289000
4290000
4291000
4292000
4293000
4294000
4295000
4296000
4297000
4298000
4299000
4300000
4301000
4302000
4303000
4304000
4305000
4306000
4307000
4308000
4309000
4310000
4311000
4312000
4313000
4314000
4315000
4316000
4317000
4318000
4319000
4320000
4321000
4322000
4323000
4324000
4325000
4326000
4327000
4328000
4329000
4330000
4331000
4332000
4333000
4334000
4335000
4336000
4337000
4338000
4339000
4340000
4341000
4342000
4343000
4344000
4345000
4346000
4347000
4348000
4349000
4350000
4351000
4352000
4353000
4354000
4355000
4356000
4357000
4358000
4359000
4360000
4361000
4362000
4363000
4364000
4365000
4366000
4367000
4368000


5270000
5271000
5272000
5273000
5274000
5275000
5276000
5277000
5278000
5279000
5280000
5281000
5282000
5283000
5284000
5285000
5286000
5287000
5288000
5289000
5290000
5291000
5292000
5293000
5294000
5295000
5296000
5297000
5298000
5299000
5300000
5301000
5302000
5303000
5304000
5305000
5306000
5307000
5308000
5309000
5310000
5311000
5312000
5313000
5314000
5315000
5316000
5317000
5318000
5319000
5320000
5321000
5322000
5323000
5324000
5325000
5326000
5327000
5328000
5329000
5330000
5331000
5332000
5333000
5334000
5335000
5336000
5337000
5338000
5339000
5340000
5341000
5342000
5343000
5344000
5345000
5346000
5347000
5348000
5349000
5350000
5351000
5352000
5353000
5354000
5355000
5356000
5357000
5358000
5359000
5360000
5361000
5362000
5363000
5364000
5365000
5366000
5367000
5368000
5369000
5370000
5371000
5372000
5373000
5374000
5375000
5376000
5377000
5378000
5379000
5380000
5381000
5382000
5383000
5384000
5385000
5386000
5387000
5388000
5389000
5390000
5391000
5392000
5393000
5394000


6295000
6296000
6297000
6298000
6299000
6300000
6301000
6302000
6303000
6304000
6305000
6306000
6307000
6308000
6309000
6310000
6311000
6312000
6313000
6314000
6315000
6316000
6317000
6318000
6319000
6320000
6321000
6322000
6323000
6324000
6325000
6326000
6327000
6328000
6329000
6330000
6331000
6332000
6333000
6334000
6335000
6336000
6337000
6338000
6339000
6340000
6341000
6342000
6343000
6344000
6345000
6346000
6347000
6348000
6349000
6350000
6351000
6352000
6353000
6354000
6355000
6356000
6357000
6358000
6359000
6360000
6361000
6362000
6363000
6364000
6365000
6366000
6367000
6368000
6369000
6370000
6371000
6372000
6373000
6374000
6375000
6376000
6377000
6378000
6379000
6380000
6381000
6382000
6383000
6384000
6385000
6386000
6387000
6388000
6389000
6390000
6391000
6392000
6393000
6394000
6395000
6396000
6397000
6398000
6399000
6400000
6401000
6402000
6403000
6404000
6405000
6406000
6407000
6408000
6409000
6410000
6411000
6412000
6413000
6414000
6415000
6416000
6417000
6418000
6419000


7320000
7321000
7322000
7323000
7324000
7325000
7326000
7327000
7328000
7329000
7330000
7331000
7332000
7333000
7334000
7335000
7336000
7337000
7338000
7339000
7340000
7341000
7342000
7343000
7344000
7345000
7346000
7347000
7348000
7349000
7350000
7351000
7352000
7353000
7354000
7355000
7356000
7357000
7358000
7359000
7360000
7361000
7362000
7363000
7364000
7365000
7366000
7367000
7368000
7369000
7370000
7371000
7372000


C:\Users\Lion\Anaconda3\envs\ml_thesis\lib\site-packages\ipykernel_launcher.py:76: RuntimeWarning: invalid value encountered in sqrt


In [8]:
h5.close()
h5f.close()
f.close()

In [5]:
import h5py
import numpy as np
h5 = h5py.File('ftr_refrmd.hdf5', 'r')
d=h5['ftr_refrmd']
tot_len=d.shape[0]
lst=0
h5.close()
on=0
h5f = h5py.File('ftr_refrmd.hdf5','r')
for i in range(0,tot_len,1000):
    mixed = h5f['ftr_refrmd'][i:i+1000]
    lst=lst+np.sum(mixed,axis=0)
    on=on+len(mixed)
tot_mean=lst/tot_len
h5f.close()
lst=0
h5f = h5py.File('ftr_refrmd.hdf5','r')
for i in range(0,tot_len,1000):
    mixed = h5f['ftr_refrmd'][i:i+1000]
    lst=lst+np.sum((mixed - tot_mean)**2,axis=0)
lst=lst/tot_len
tot_std=np.sqrt(lst)
h5f.close()


OSError: Unable to create file (unable to truncate a file which is already open)

In [9]:
f = h5py.File('ftr_scaled.hdf5', 'w')
dd = f.create_dataset('ftr_scaled', (0,1799),maxshape=(None,1799), dtype='f', chunks=True)
h5f = h5py.File('ftr_refrmd.hdf5','r')
for i in range(0,tot_len,1000):
    mixed = h5f['ftr_refrmd'][i:i+1000]
    scaled=(mixed-tot_mean)/tot_std
    dd.resize(dd.shape[0]+scaled.shape[0], axis=0)   
    dd[-1*scaled.shape[0]:] = scaled

In [7]:
np.savetxt('mean.txt',tot_mean)
np.savetxt('std.txt',tot_std)

In [43]:
import h5py
import numpy as np
h5 = h5py.File('ftr_refrmd.hdf5', 'r')
d=h5['ftr_refrmd']
tot_len=d.shape[0]
lst=0
h5.close()
on=0
h5f = h5py.File('ftr_refrmd.hdf5','r')
for i in range(0,tot_len,1000):
    mixed = h5f['ftr_refrmd'][i:i+1000]
    lst=lst+np.sum(mixed,axis=0)
    on=on+len(mixed)
tot_mean=lst/tot_len
h5f.close()
lst=0
h5f = h5py.File('ftr_refrmd.hdf5','r')
for i in range(0,tot_len,1000):
    mixed = h5f['ftr_refrmd'][i:i+1000]
    lst=lst+np.sum(mixed - tot_mean**2,axis=0)
lst=lst/tot_len
tot_std=np.sqrt(lst)
h5f.close()


C:\Users\ASUS\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in sqrt


In [ ]:
f.close()
f = h5py.File('ftr_scaled.hdf5', 'w')
dd = f.create_dataset('ftr_scaled', (0,1799),maxshape=(None,1799), dtype='f', chunks=True)
h5f = h5py.File('ftr_refrmd.hdf5','r')
for i in range(0,tot_len,1000):
    mixed = h5f['ftr_refrmd'][i:i+1000]
    scaled=(mixed-tot_mean)/tot_std
    dd.resize(dd.shape[0]+scaled.shape[0], axis=0)   
    dd[-1*scaled.shape[0]:] = scaled

In [32]:
on

7183134

In [34]:
h5f = h5py.File('ftr_refrmd.hdf5','r')
mixed = h5f['ftr_refrmd'][0]
h5f.close()
np.mean(mixed-tot_mean)

-0.3306819626452817

In [77]:
import h5py
h5f = h5py.File('mixed_log2.hdf5','r')
mixed = h5f['mixed_log2'][0:110]
h5f.close()
print('loaded')
from sklearn import preprocessing
import numpy as np
scaler = preprocessing.StandardScaler().fit(mixed.T)
from sklearn.externals import joblib
scaler_filename = "standard_scaler"
joblib.dump(scaler, scaler_filename) 
#inverse_transform(X, copy=None)

loaded


In [22]:
# And now to load...
from sklearn.externals import joblib
scaler = joblib.load('standard_scaler')
import h5py
h5f = h5py.File('mixed_log2.hdf5','r')
mixed = h5f['mixed_log2'][0:1000]
h5f.close()
print('loaded')
scaled_mix=scaler.transform(mixed.T)


In [28]:
import h5py
h5f = h5py.File('scaled_mix.h5', 'w')
h5f.create_dataset('scaled_mix', data=scaled_mix.T)
h5f.close()


In [66]:
f.close()

In [4]:
import h5py
import math as m
import os
import librosa
import numpy as np
orig_path=os.getcwd()
wav=[]
l=os.listdir(os.path.join(orig_path,'single_dataset_16'))
l.sort(key=int)
filename1 = l[1]
i=0
w=7
num_of_snrs=21
num_of_speakers=6
files = os.listdir(os.path.join(orig_path,'single_dataset_16',filename1))
f.close()
f = h5py.File('single_dataset_log_16.hdf5', 'w')
d = f.create_dataset('single_dataset_log_16', (0,257),maxshape=(None,257), dtype='f', chunks=True)
#in this section the features for clean voices are extracted.
for voice in files:
    #sr, wav_data = wavfile.read(os.path.join(orig_path,'single dataset',filename1,filename2)+"\\"+voice, )
    #sr, wav_data = wavfile.read(os.path.join(orig_path,'mixed2_with24')+"\\"+filename)
    wav_data, sr = librosa.load(os.path.join(orig_path,'single_dataset_16',filename1,voice), sr=16000) 
    framed_data=librosa.core.stft(wav_data, n_fft=512, hop_length=256, win_length=512, window='hann')
    #frame length = win_length * freq *1000(ms)
    abslt=np.absolute(framed_data)**2
    dft_signal=np.log10(abslt+1e-7*np.ones(np.shape(abslt)))
#     for i in range(m.floor(w/2)):
#         dft_signal=np.insert(dft_signal,0,0,axis = 1)
    #data_phase=np.angle(framed_data)
    for i in range(num_of_snrs*num_of_speakers):
        d.resize(d.shape[0]+dft_signal.shape[1], axis=0)   
        d[-1*dft_signal.shape[1]:] = dft_signal.T
    #np.savetxt('single_dataset_log_16\\abs'+'2'+'_'+voice+'.txt', dft_signal, delimiter=',')
    #np.savetxt('single_dataset_phase_features_256\\phase'+'1'+'_'+str(i+1)+'.txt', data_phase, delimiter=',')
f.close()

    


In [13]:
import h5py
h5f = h5py.File('ftr_refrmd.hdf5','r')
single = h5f['ftr_refrmd'][-1]
h5f.close()


In [4]:
import h5py
h5f = h5py.File('single_dataset_log_16.hdf5','r')
single = h5f['single_dataset_log_16'][:]
h5f.close()


In [6]:
single.shape[0]

7372134

In [29]:
del scaled_mix

In [11]:
import h5py
h5f = h5py.File('mixed_log2.hdf5','r')
ftr = h5f['mixed_log2'][0:]
h5f.close()

In [14]:
import numpy as np
ftr_refrmd = np.zeros((983134,257*7))

In [17]:
ftr.shape

(7372134, 257)

In [52]:
import math as m
import numpy as np
import h5py
w = 7
ftr_refrmd=[]
for i in range(m.floor(w/2)):
    p=-m.floor(w/2)
    temp = []
    for j in range(w):
        if np.all(ftr[i]==0):
            break
        #if j<m.floor(w/2)-i+1:
            #temp.extend(np.zeros(ftr[0].shape))
        #else:
        #if j>m.floor(w/2)-i+1:
        temp.extend(ftr[i+p+j])
        #temp.extend(ftr[j-(m.floor(w/2)-i+1)])
    if np.any(temp!=[]):
        ftr_refrmd.append(temp)
f.close()
f = h5py.File('ftr_refrmd.hdf5', 'w')
d = f.create_dataset('ftr_refrmd', (0,257*7),maxshape=(None,257*7), dtype='f', chunks=True)
for i in range(m.floor(w/2),ftr.shape[0]-m.floor(w/2)-1):
    k=-m.floor(w/2)
    temp = []
    for j in range(w):
        if np.all(ftr[i]==0):
            break
        temp.extend(ftr[i+k])
        k=k+1
    if np.any(temp!=[]):
        ftr_refrmd.append(temp)
    if len(ftr_refrmd)>100000 or i==ftr.shape[0]-m.floor(w/2)-2:
        d.resize(d.shape[0]+len(ftr_refrmd), axis=0)   
        d[-1*len(ftr_refrmd):] = ftr_refrmd
        ftr_refrmd=[]
    if i%1000==0:
        print(i)
for i in range(ftr.shape[0]-m.floor(w/2)-1,ftr.shape[0]):
    p=-m.floor(w/2)
    temp = []
    for j in range(w):
        if np.all(ftr[i]==0):
            break
        if j>m.floor(w/2)+(ftr.shape[0]-i)-1:
            temp.extend(np.zeros(ftr[ftr.shape[0]-1].shape))
        else:
            temp.extend(ftr[i+p+j])
    if np.any(temp!=[]):
        ftr_refrmd.append(temp)
d.resize(d.shape[0]+len(ftr_refrmd), axis=0)   
d[-1*len(ftr_refrmd):] = ftr_refrmd
ftr_refrmd=[]

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
15

1165000
1166000
1167000
1168000
1169000
1170000
1171000
1172000
1173000
1174000
1175000
1176000
1177000
1178000
1179000
1180000
1181000
1182000
1183000
1184000
1185000
1186000
1187000
1188000
1189000
1190000
1191000
1192000
1193000
1194000
1195000
1196000
1197000
1198000
1199000
1200000
1201000
1202000
1203000
1204000
1205000
1206000
1207000
1208000
1209000
1210000
1211000
1212000
1213000
1214000
1215000
1216000
1217000
1218000
1219000
1220000
1221000
1222000
1223000
1224000
1225000
1226000
1227000
1228000
1229000
1230000
1231000
1232000
1233000
1234000
1235000
1236000
1237000
1238000
1239000
1240000
1241000
1242000
1243000
1244000
1245000
1246000
1247000
1248000
1249000
1250000
1251000
1252000
1253000
1254000
1255000
1256000
1257000
1258000
1259000
1260000
1261000
1262000
1263000
1264000
1265000
1266000
1267000
1268000
1269000
1270000
1271000
1272000
1273000
1274000
1275000
1276000
1277000
1278000
1279000
1280000
1281000
1282000
1283000
1284000
1285000
1286000
1287000
1288000
1289000


2190000
2191000
2192000
2193000
2194000
2195000
2196000
2197000
2198000
2199000
2200000
2201000
2202000
2203000
2204000
2205000
2206000
2207000
2208000
2209000
2210000
2211000
2212000
2213000
2214000
2215000
2216000
2217000
2218000
2219000
2220000
2221000
2222000
2223000
2224000
2225000
2226000
2227000
2228000
2229000
2230000
2231000
2232000
2233000
2234000
2235000
2236000
2237000
2238000
2239000
2240000
2241000
2242000
2243000
2244000
2245000
2246000
2247000
2248000
2249000
2250000
2251000
2252000
2253000
2254000
2255000
2256000
2257000
2258000
2259000
2260000
2261000
2262000
2263000
2264000
2265000
2266000
2267000
2268000
2269000
2270000
2271000
2272000
2273000
2274000
2275000
2276000
2277000
2278000
2279000
2280000
2281000
2282000
2283000
2284000
2285000
2286000
2287000
2288000
2289000
2290000
2291000
2292000
2293000
2294000
2295000
2296000
2297000
2298000
2299000
2300000
2301000
2302000
2303000
2304000
2305000
2306000
2307000
2308000
2309000
2310000
2311000
2312000
2313000
2314000


3215000
3216000
3217000
3218000
3219000
3220000
3221000
3222000
3223000
3224000
3225000
3226000
3227000
3228000
3229000
3230000
3231000
3232000
3233000
3234000
3235000
3236000
3237000
3238000
3239000
3240000
3241000
3242000
3243000
3244000
3245000
3246000
3247000
3248000
3249000
3250000
3251000
3252000
3253000
3254000
3255000
3256000
3257000
3258000
3259000
3260000
3261000
3262000
3263000
3264000
3265000
3266000
3267000
3268000
3269000
3270000
3271000
3272000
3273000
3274000
3275000
3276000
3277000
3278000
3279000
3280000
3281000
3282000
3283000
3284000
3285000
3286000
3287000
3288000
3289000
3290000
3291000
3292000
3293000
3294000
3295000
3296000
3297000
3298000
3299000
3300000
3301000
3302000
3303000
3304000
3305000
3306000
3307000
3308000
3309000
3310000
3311000
3312000
3313000
3314000
3315000
3316000
3317000
3318000
3319000
3320000
3321000
3322000
3323000
3324000
3325000
3326000
3327000
3328000
3329000
3330000
3331000
3332000
3333000
3334000
3335000
3336000
3337000
3338000
3339000


4241000
4242000
4243000
4244000
4245000
4246000
4247000
4248000
4249000
4250000
4251000
4252000
4253000
4254000
4255000
4256000
4257000
4258000
4259000
4260000
4261000
4262000
4263000
4264000
4265000
4266000
4267000
4268000
4269000
4270000
4271000
4272000
4273000
4274000
4275000
4276000
4277000
4278000
4279000
4280000
4281000
4282000
4283000
4284000
4285000
4286000
4287000
4288000
4289000
4290000
4291000
4292000
4293000
4294000
4295000
4296000
4297000
4298000
4299000
4300000
4301000
4302000
4303000
4304000
4305000
4306000
4307000
4308000
4309000
4310000
4311000
4312000
4313000
4314000
4315000
4316000
4317000
4318000
4319000
4320000
4321000
4322000
4323000
4324000
4325000
4326000
4327000
4328000
4329000
4330000
4331000
4332000
4333000
4334000
4335000
4336000
4337000
4338000
4339000
4340000
4341000
4342000
4343000
4344000
4345000
4346000
4347000
4348000
4349000
4350000
4351000
4352000
4353000
4354000
4355000
4356000
4357000
4358000
4359000
4360000
4361000
4362000
4363000
4364000
4365000


5266000
5267000
5268000
5269000
5270000
5271000
5272000
5273000
5274000
5275000
5276000
5277000
5278000
5279000
5280000
5281000
5282000
5283000
5284000
5285000
5286000
5287000
5288000
5289000
5290000
5291000
5292000
5293000
5294000
5295000
5296000
5297000
5298000
5299000
5300000
5301000
5302000
5303000
5304000
5305000
5306000
5307000
5308000
5309000
5310000
5311000
5312000
5313000
5314000
5315000
5316000
5317000
5318000
5319000
5320000
5321000
5322000
5323000
5324000
5325000
5326000
5327000
5328000
5329000
5330000
5331000
5332000
5333000
5334000
5335000
5336000
5337000
5338000
5339000
5340000
5341000
5342000
5343000
5344000
5345000
5346000
5347000
5348000
5349000
5350000
5351000
5352000
5353000
5354000
5355000
5356000
5357000
5358000
5359000
5360000
5361000
5362000
5363000
5364000
5365000
5366000
5367000
5368000
5369000
5370000
5371000
5372000
5373000
5374000
5375000
5376000
5377000
5378000
5379000
5380000
5381000
5382000
5383000
5384000
5385000
5386000
5387000
5388000
5389000
5390000


6292000
6293000
6294000
6295000
6296000
6297000
6298000
6299000
6300000
6301000
6302000
6303000
6304000
6305000
6306000
6307000
6308000
6309000
6310000
6311000
6312000
6313000
6314000
6315000
6316000
6317000
6318000
6319000
6320000
6321000
6322000
6323000
6324000
6325000
6326000
6327000
6328000
6329000
6330000
6331000
6332000
6333000
6334000
6335000
6336000
6337000
6338000
6339000
6340000
6341000
6342000
6343000
6344000
6345000
6346000
6347000
6348000
6349000
6350000
6351000
6352000
6353000
6354000
6355000
6356000
6357000
6358000
6359000
6360000
6361000
6362000
6363000
6364000
6365000
6366000
6367000
6368000
6369000
6370000
6371000
6372000
6373000
6374000
6375000
6376000
6377000
6378000
6379000
6380000
6381000
6382000
6383000
6384000
6385000
6386000
6387000
6388000
6389000
6390000
6391000
6392000
6393000
6394000
6395000
6396000
6397000
6398000
6399000
6400000
6401000
6402000
6403000
6404000
6405000
6406000
6407000
6408000
6409000
6410000
6411000
6412000
6413000
6414000
6415000
6416000


7317000
7318000
7319000
7320000
7321000
7322000
7323000
7324000
7325000
7326000
7327000
7328000
7329000
7330000
7331000
7332000
7333000
7334000
7335000
7336000
7337000
7338000
7339000
7340000
7341000
7342000
7343000
7344000
7345000
7346000
7347000
7348000
7349000
7350000
7351000
7352000
7353000
7354000
7355000
7356000
7357000
7358000
7359000
7360000
7361000
7362000
7363000
7364000
7365000
7366000
7367000
7368000
7369000
7370000
7371000
7372000


In [51]:
ftr[6]

array([ 0.7070756 ,  1.1297452 ,  0.99927664,  0.7218095 ,  0.3626349 ,
       -0.26529425, -0.51360375, -0.6183383 , -0.4936812 , -0.28964832,
       -0.11785527, -0.05113456, -0.05586933, -0.13723312, -0.23845498,
       -0.3437945 , -0.565178  , -0.79476017, -0.9641734 , -1.2127057 ,
       -1.6020828 , -1.8547798 , -1.5866703 , -1.4565933 , -1.53075   ,
       -1.6472334 , -1.6913518 , -1.6796035 , -1.7875504 , -1.8356729 ,
       -1.7477174 , -1.7175528 , -1.9353228 , -2.3350923 , -2.7471178 ,
       -2.512575  , -2.248996  , -2.0597937 , -1.7719635 , -1.5882171 ,
       -1.4581786 , -1.40668   , -1.3930643 , -1.4578096 , -1.5470549 ,
       -1.5715003 , -1.4820395 , -1.4494987 , -1.4181166 , -1.3916979 ,
       -1.4421563 , -1.5223688 , -1.7184484 , -1.9396169 , -2.1304336 ,
       -2.3632662 , -2.4623733 , -2.3663127 , -2.162645  , -2.0569775 ,
       -1.967594  , -1.8637985 , -1.6720158 , -1.4775913 , -1.4155997 ,
       -1.4015068 , -1.4585706 , -1.5053412 , -1.5103936 , -1.54

In [50]:
ftr_refrmd[0][-257:]

[0.7070756,
 1.1297452,
 0.99927664,
 0.7218095,
 0.3626349,
 -0.26529425,
 -0.51360375,
 -0.6183383,
 -0.4936812,
 -0.28964832,
 -0.11785527,
 -0.05113456,
 -0.055869326,
 -0.13723312,
 -0.23845498,
 -0.3437945,
 -0.565178,
 -0.79476017,
 -0.9641734,
 -1.2127057,
 -1.6020828,
 -1.8547798,
 -1.5866703,
 -1.4565933,
 -1.53075,
 -1.6472334,
 -1.6913518,
 -1.6796035,
 -1.7875504,
 -1.8356729,
 -1.7477174,
 -1.7175528,
 -1.9353228,
 -2.3350923,
 -2.7471178,
 -2.512575,
 -2.248996,
 -2.0597937,
 -1.7719635,
 -1.5882171,
 -1.4581786,
 -1.40668,
 -1.3930643,
 -1.4578096,
 -1.5470549,
 -1.5715003,
 -1.4820395,
 -1.4494987,
 -1.4181166,
 -1.3916979,
 -1.4421563,
 -1.5223688,
 -1.7184484,
 -1.9396169,
 -2.1304336,
 -2.3632662,
 -2.4623733,
 -2.3663127,
 -2.162645,
 -2.0569775,
 -1.967594,
 -1.8637985,
 -1.6720158,
 -1.4775913,
 -1.4155997,
 -1.4015068,
 -1.4585706,
 -1.5053412,
 -1.5103936,
 -1.5468695,
 -1.5891609,
 -1.6694322,
 -1.8742436,
 -2.2332778,
 -2.5928373,
 -2.8199828,
 -3.0207024,
 -

In [3]:
import h5py
h5f = h5py.File('single_dataset_log_16.hdf5','r')
single_dataset = h5f['single_dataset_log_16'][0:]
h5f.close()

In [12]:
single_dataset.shape

(7183134, 257)

In [5]:
 np.arange(0,21*6*,2)

(257,)

In [55]:
orig_path=os.getcwd()
from natsort import natsorted
# import h5py
# h5f = h5py.File('single_dataset_log_16.hdf5','r')
# single_dataset = h5f['single_dataset_log_16'][0:]
# h5f.close()
target1 = single_dataset[0].T 
for i in 
target = single_dataset[0].T
#target=np.loadtxt('single_dataset_log_16\\'+'abs1_1.txt',delimiter=',').T
#target1=np.loadtxt('single_dataset_log_16\\'+'abs1_1.txt',delimiter=',').T
for i in range(21*6):
    target1=np.concatenate((target1, target),axis=0)
print(target1.shape)
j=1
a=os.listdir(os.path.join(orig_path,'single_dataset_log_16'))
a=natsorted(a)
for filename in a[1:]:
    f=np.loadtxt('single_dataset_log_16\\'+filename,delimiter=',')
    print(f.shape)
    for i in range(600):
        target1=np.concatenate((target1,f.T),axis=0)
    #print(j)
    print(target1.shape)
    if j==4:
        break
    j=j+1
print(target1.shape)

SyntaxError: invalid syntax (<ipython-input-55-050848438f08>, line 8)

In [1]:
import numpy as np
import os
from scipy.io import wavfile
import librosa
import csv
import math as m
from natsort import natsorted
import h5py
#in this section the features for mixed voices are extracted.
orig_path=os.getcwd()
i = 1
a = os.listdir(os.path.join(orig_path,'sec_mixed2_6'))
a = natsorted(a)

C:\Users\ASUS\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
a

['mixed_bbaf3p_1_0_pbbc5s_1_251.wav',
 'mixed_bbaf3p_1_0_pbwj1s_3_273.wav',
 'mixed_bbaf3p_1_0_prit6p_21_354.wav',
 'mixed_bbaf3p_1_0_pwbhzn_31_355.wav',
 'mixed_bbaf3p_1_0_sbij6s_34_385.wav',
 'mixed_bbaf3p_1_0_swwo6p_9_498.wav',
 'mixed_bbaf3p_1_1_briv8p_31_89.wav',
 'mixed_bbaf3p_1_1_bwbt4n_3_104.wav',
 'mixed_bbaf3p_1_1_lbbc2p_21_132.wav',
 'mixed_bbaf3p_1_1_lrah3n_34_191.wav',
 'mixed_bbaf3p_1_1_lrik4p_1_201.wav',
 'mixed_bbaf3p_1_1_sgbv2p_9_403.wav',
 'mixed_bbaf3p_1_2_bbwl9a_9_32.wav',
 'mixed_bbaf3p_1_2_bwba4n_1_97.wav',
 'mixed_bbaf3p_1_2_lbij4n_3_135.wav',
 'mixed_bbaf3p_1_2_lgij4n_21_176.wav',
 'mixed_bbaf3p_1_2_lrwv3s_31_211.wav',
 'mixed_bbaf3p_1_2_sgir8s_34_424.wav',
 'mixed_bbaf3p_1_3_brbp9p_34_67.wav',
 'mixed_bbaf3p_1_3_brbx6p_31_80.wav',
 'mixed_bbaf3p_1_3_lgaz6n_1_159.wav',
 'mixed_bbaf3p_1_3_pgbd7a_9_272.wav',
 'mixed_bbaf3p_1_3_pwbu8n_21_378.wav',
 'mixed_bbaf3p_1_3_sgib7s_3_417.wav',
 'mixed_bbaf3p_1_4_bbilzn_3_21.wav',
 'mixed_bbaf3p_1_4_bgid9a_31_50.wav',
 'mixe